In [ ]:

import numpy as np

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Metrics
from sklearn.metrics import confusion_matrix

# Suppressing warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import os
import zipfile

# Define the path to the archive and the extraction directory
archive_path = 'data/archive.zip'
extract_dir = 'data'

# Check if the archive exists
if os.path.exists(archive_path):
    # Open the ZIP file
    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        # Extract all contents to the specified directory
        zip_ref.extractall(extract_dir)
    print(f"Successfully extracted contents of {archive_path} to {extract_dir}")
else:
    print(f"Error: {archive_path} not found")


Successfully extracted contents of data/archive.zip to data


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
imgSize = 112

transformer = transforms.Compose([
    transforms.RandomRotation(degrees= 20),
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.RandomVerticalFlip(p = 0.3),
    transforms.Resize(size = (imgSize, imgSize), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

valTransformer = transforms.Compose([
    transforms.Resize(size = (imgSize, imgSize), antialias=True),
    transforms.CenterCrop(size = (imgSize, imgSize)),

    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [ ]:
model= models.efficientnet_v2_1(weights= 'DEFAULT')
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)

In [ ]:
model = nn.DataParallel(model).to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = ReduceLROnPlateau(optimizer, threshold = 0.01, factor = 0.1, patience = 3, 
                              min_lr = 1e-5, verbose = True)

In [ ]:
patience = 5
minDelta = 0.01
currentPatience = 0
bestLoss = float('inf')